In [ ]:
# cands is the answer from the QA dataset
# ref1 and ref2 are outputs from Gpt3.5 and Llama2

# Question: Will capital gains affect my tax bracket?
cands = ["I'm not sure where you are, but in the United States capital gains are taxed at a lower rate than other types of income. On the 1040, captial gains income is separated from earned income, and income tax is calculated just on earned income. Then capital gains tax is calculated on capital gains income, and then added to income tax afterward."]
ref1 = ["As a Wealth Manager, I can assure you that capital gains can indeed impact your tax bracket. When you sell an investment for a profit, the gain is subject to capital gains tax, which can range from 0% to 20%, depending on your income tax bracket and the length of time you've held the investment. If your capital gains push your income into a higher tax bracket, you may end up paying a higher tax rate on some of your gains. However, it's worth noting that the tax implications of capital gains can be complex, and there may be strategies to minimize your tax liability, such as holding onto your investments for at least a year to qualify for long-term capital gains treatment. It's always a good idea to consult with a tax professional or financial advisor to understand the specific tax implications of your investment decisions."]
ref2 = ["Whether capital gains affect your tax bracket depends on two factors: gains type and size. Short-term gains (held less than a year) are taxed at your ordinary income rate, potentially bumping you to a higher bracket. Long-term gains (held over a year) usually have lower rates (0%, 15%, or 20%), but large gains could still push you over the threshold. Remember, it's individual, so consulting a tax professional for specific guidance is crucial."]

# BERT Score

In [3]:
from bert_score import score


Bad key "text.kerning_factor" on line 4 in
C:\Users\Dibyanshu Patnaik\.conda\envs\jupyter\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [6]:
P1, R1, F11 = score(cands, ref1, lang="en", verbose=True)
P2, R2, F12 = score(cands, ref2, lang="en", verbose=True)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.64 seconds, 0.61 sentences/sec


In [7]:
print(F11, F12)

tensor([0.8489]) tensor([0.8470])


# Sentence Transformers

In [14]:
from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer("all-MiniLM-L6-v2")
model = SentenceTransformer("stsb-roberta-large")

In [15]:
embeddings1 = model.encode(cands, convert_to_tensor=True)
embeddings2 = model.encode(ref1, convert_to_tensor=True)
embeddings3 = model.encode(ref2, convert_to_tensor=True)

In [16]:
cosine_scores1 = util.cos_sim(embeddings1, embeddings2)
cosine_scores2 = util.cos_sim(embeddings1, embeddings3)

print(cosine_scores1, cosine_scores2)

tensor([[0.6036]]) tensor([[0.6389]])


# Jaccard Similarity

In [17]:
def jaccard_similarity(x,y):
  intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
  union_cardinality = len(set.union(*[set(x), set(y)]))
  return intersection_cardinality/float(union_cardinality)

print(jaccard_similarity([text.lower().split(" ") for text in cands][0],[text.lower().split(" ") for text in ref1][0]))
print(jaccard_similarity([text.lower().split(" ") for text in cands][0],[text.lower().split(" ") for text in ref2][0]))

0.13043478260869565
0.20238095238095238


In [21]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [24]:
inputs = tokenizer(cands, return_tensors = "pt") #Answer in the QA Dataset
loss = model(input_ids = inputs["input_ids"], labels = inputs["input_ids"]).loss
ppl = torch.exp(loss)
print(ppl)

inputs = tokenizer(ref1, return_tensors = "pt")
loss = model(input_ids = inputs["input_ids"], labels = inputs["input_ids"]).loss
ppl = torch.exp(loss)
print(ppl)

inputs = tokenizer(ref2, return_tensors = "pt")
loss = model(input_ids = inputs["input_ids"], labels = inputs["input_ids"]).loss
ppl = torch.exp(loss)
print(ppl)

tensor(22.8634, grad_fn=<ExpBackward0>)
tensor(8.9369, grad_fn=<ExpBackward0>)
tensor(21.5526, grad_fn=<ExpBackward0>)


# BLEU and ROUGE

In [51]:
from sacrebleu.metrics import BLEU
bleu_scorer = BLEU()


score1 = bleu_scorer.sentence_score(
    hypothesis=cands[0],
    references=ref1,
)
score2 = bleu_scorer.sentence_score(
    hypothesis=cands[0],
    references=ref2,
)
print("BLEU")
print(score1.score/100)
print(score2.score/100)

from rouge import Rouge
rouge_scorer = Rouge()

score1 = rouge_scorer.get_scores(
    hyps=cands[0],
    refs=ref1[0],
)
score2 = rouge_scorer.get_scores(
    hyps=cands[0],
    refs=ref2[0],
)
print("ROUGE")
print(score1[0]["rouge-l"]["f"])
print(score2[0]["rouge-l"]["f"])

It is recommended to enable `effective_order` for sentence-level BLEU.
It is recommended to enable `effective_order` for sentence-level BLEU.


BLEU
0.011906943486511454
0.02873622116900213
ROUGE
0.2307692263952663
0.2772277179139301


# TFIDF

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Combine the texts into a single list
all_texts = cands + ref1 + ref2

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit the vectorizer on all texts and transform each text to its TF-IDF representation
tfidf_matrix = tfidf_vectorizer.fit_transform(all_texts)

# Separate the TF-IDF representations for each text
tfidf_matrix_text1 = tfidf_matrix[:len(cands)]
tfidf_matrix_text2 = tfidf_matrix[len(cands):len(cands)+len(ref1)]
tfidf_matrix_text3 = tfidf_matrix[len(ref1)+len(ref1):]

# Calculate cosine similarity using TF-IDF representations
cosine_sim_tfidf1_2 = cosine_similarity(tfidf_matrix_text1, tfidf_matrix_text2)
cosine_sim_tfidf1_3 = cosine_similarity(tfidf_matrix_text1, tfidf_matrix_text3)

print("Cosine Similarity using TF-IDF between embeddings 1 and 2:", cosine_sim_tfidf1_2[0][0])
print("Cosine Similarity using TF-IDF between embeddings 1 and 3:", cosine_sim_tfidf1_3[0][0])

Cosine Similarity using TF-IDF between embeddings 1 and 2: 0.3285035408811078
Cosine Similarity using TF-IDF between embeddings 1 and 3: 0.31417397867087804
